In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from dotenv import load_dotenv
import edurel.utils.question_man as qm
import edurel.utils.conversation as conv
import edurel.utils.duckdb as ddb
import edurel.utils.sql_agent as sa


load_dotenv() 
BASE_DIR = os.getenv("BASE_DIR")
QUESTIONS_DIR = f"{BASE_DIR}/questions"
CONFIG_DIR = f"{BASE_DIR}/config"
LOG_DIR = f"{BASE_DIR}/logs/sql"


/home/iclassen/.py1/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


# Database

In [3]:
db = ddb.Db.mem("company_en")
db.config_exe(f"{CONFIG_DIR}/databases/company_en.yaml")

# Questions

In [4]:
qman = qm.QuestionMan(f"{QUESTIONS_DIR}/company_en.txt")

In [ ]:
sqlagent = sa.SQLAgent(
    db,
    conv.Client.STATS, 
    conv.LLMName.QWEN3CODER,
    fk_spec_file_path=f"{CONFIG_DIR}/fk_specs/company_en.yaml",
    rem_tags_spec_file_path=f"{CONFIG_DIR}/omit_tags_specs/dbstuff.yaml",
    log_dir=LOG_DIR,
)

In [10]:
conv.LLM_NAME_STRINGS[conv.LLMName.QWEN3CODER]

'qwen3-coder-480b-a35b-instruct'

# Text2SQL

In [12]:
for q in qman[:10]:
    print(q[0])
    sqlagent.text2sql(q[0], q[1], autolog=True)
    sqlagent.conversation.delete_slice(2)


entry_year
entry_year_dept_14
hireyears_dept_14
employees_without_bonus
employees_with_bonus
total_income_per_employe_dept_17
salary_band_per_employee
employees_with_orgunit_name
employees_per_project
parent_child_org


# Testing

In [ ]:
sqlagent.text2sql(qman[0][0], qman[0][1])

In [12]:
sqlagent.text2sql(qman[1][0], qman[1][1])


In [15]:
sqlagent.conversation.delete_slice(2)


3

In [ ]:
sqlagent.conversation.display(lastn_only=4)

In [7]:
sqlagent.log(qman[0][0], "v1")

In [ ]:
print(qman[0][1])

In [ ]:
for q in qman[:]:
    print(f"---tag: {q[0]}")
    print(q[1])